In [11]:
from txgnn import TxData, TxGNN, TxEval
import pandas as pd
import dgl


In [4]:
# check pandas version
print(pd.__version__)
print(dgl.__version__)


1.5.3
0.9.1


In [9]:
%reload_ext autoreload
%autoreload 2


In [13]:
tx_data = TxData('./data')
tx_data.prepare_split(split='random', seed=1)
tx_gnn = TxGNN(data=tx_data, weight_bias_track=False, proj_name='TxGNN', exp_name='TxGNN', device='cuda:0')


Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
Done!


In [14]:
tx_gnn.load_pretrained('/Users/shloknatarajan/stanford/research/daneshjou/TxGNN-Burkitts/checkpoints_all_seeds/TxGNN_1_random')

anatomy_anatomy
rev_anatomy_protein_absent
rev_anatomy_protein_present
bioprocess_bioprocess
rev_bioprocess_protein
rev_exposure_bioprocess
cellcomp_cellcomp
rev_cellcomp_protein
rev_exposure_cellcomp
disease_disease
disease_phenotype_negative
disease_phenotype_positive
rev_contraindication
rev_disease_protein
rev_exposure_disease
rev_indication
rev_off-label use
contraindication
drug_drug
drug_effect
drug_protein
indication
off-label use
phenotype_phenotype
rev_disease_phenotype_negative
rev_disease_phenotype_positive
rev_drug_effect
rev_phenotype_protein
exposure_bioprocess
exposure_cellcomp
exposure_disease
exposure_exposure
exposure_molfunc
exposure_protein
anatomy_protein_absent
anatomy_protein_present
bioprocess_protein
cellcomp_protein
disease_protein
molfunc_protein
pathway_protein
phenotype_protein
protein_protein
rev_drug_protein
rev_exposure_protein
molfunc_molfunc
rev_exposure_molfunc
rev_molfunc_protein
pathway_pathway
rev_pathway_protein


DGLError: [13:36:13] /tmp/dgl_src/src/graph/heterograph.cc:120: Check failed: !rel_graphs.empty(): Empty heterograph is not allowed.
Stack trace:
  [bt] (0) 1   libdgl.dylib                        0x000000015d0b329c dmlc::LogMessageFatal::~LogMessageFatal() + 116
  [bt] (1) 2   libdgl.dylib                        0x000000015d3f44a0 dgl::HeteroGraph::HeteroGraph(std::__1::shared_ptr<dgl::GraphInterface>, std::__1::vector<std::__1::shared_ptr<dgl::BaseHeteroGraph>, std::__1::allocator<std::__1::shared_ptr<dgl::BaseHeteroGraph>>> const&, std::__1::vector<long long, std::__1::allocator<long long>> const&) + 1596
  [bt] (2) 3   libdgl.dylib                        0x000000015d3c6958 dgl::CreateHeteroGraph(std::__1::shared_ptr<dgl::GraphInterface>, std::__1::vector<std::__1::shared_ptr<dgl::BaseHeteroGraph>, std::__1::allocator<std::__1::shared_ptr<dgl::BaseHeteroGraph>>> const&, std::__1::vector<long long, std::__1::allocator<long long>> const&) + 96
  [bt] (3) 4   libdgl.dylib              

In [10]:
mappings = data.retrieve_id_mapping()
id2name_drug = mappings['id2name_drug']
idx2id_drug = mappings['idx2id_drug']